In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt

In [6]:
def plot_psi(section, use_density):
    # read stremfunction
    path = f"single_lats/{section}_sf"
    path = path + "_density.pickle" if use_density else path + "_depth.pickle"
    infile = open(path, "rb")
    psi = pickle.load(infile); infile.close()
    psi = psi[:, 1, :].mean(axis=1)
    if use_density: psi = psi[::-1]

    # read density/depth range for axis ticks
    if use_density:
        path = f"single_lats/{section}_density_range.pickle" 
    else:
        path = f"single_lats/depth_range.pickle"
    infile = open(path, "rb")
    d = pickle.load(infile); infile.close()
    d = [round(x, 2) for x in d]
    d = d[::-1]

    start = np.argmax(psi < 0.)
    sign = lambda x: (x > 0)*2 - 1
    end = np.argmax(sign(psi[start:]) != sign(psi[start])) + start
    start = 0
    abyssal = f"Abyssal Cell Range: [{str(round(d[start], 2))}, {str(round(d[end], 2))}]"

    fig, ax = plt.subplots(figsize=(4, 5))
    ax.plot(psi, np.arange(len(d)), linestyle="-", alpha=0.7)
    ax.set_yticks(np.arange(0, len(d), 10), d[::10])
    xlabel = "$\\Psi(\\sigma_2)$ [SV]" if use_density else "$\\Psi$(Z) [Sv]"
    ax.set_xlabel(xlabel)
    ylabel = "$\\sigma_2$ [kg/m^3]" if use_density else "Z [m]"
    ax.set_ylabel(ylabel)
    ax.set_title(section + "\n" + abyssal, fontsize=10)
    ax.vlines(0, 0, len(d), color="black", linestyle="--", alpha=0.2)
    ax.set_xlim(min(psi)-1, max(psi)+1)
    ax.fill_between(np.arange(min(psi)-2, max(psi)+2), start, end, color="black", alpha=0.2)
    plt.tight_layout()
    plot_title = f"{section}_density" if use_density else f"{section}_depth"
    plt.savefig(f"plots/{plot_title}.png", dpi=400)
    plt.close()

In [7]:
sections = ["26N", "30S", "55S", "60S"]
for section in sections:
    plot_psi(section, True)
    plot_psi(section, False)